In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    directory="dataset",
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    directory="dataset",
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=True
)

Found 20000 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Evita overfitting
    Dense(1, activation='sigmoid')  # Para clasificación binaria
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [5]:
# Entrenamiento del modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10  # puedes ajustar según tu tiempo y dataset
)


Epoch 1/10
209/625 [=========>....................] - ETA: 2:56 - loss: 0.6906 - accuracy: 0.5529 

C:\Users\juanito\AppData\Roaming\Python\Python310\site-packages\PIL\TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


625/625 [==============================] - 325s 519ms/step - loss: 0.6325 - accuracy: 0.6346 - val_loss: 0.5465 - val_accuracy: 0.7183
Epoch 2/10
625/625 [==============================] - 300s 480ms/step - loss: 0.5113 - accuracy: 0.7530 - val_loss: 0.4753 - val_accuracy: 0.7871
Epoch 3/10
625/625 [==============================] - 298s 477ms/step - loss: 0.4418 - accuracy: 0.7965 - val_loss: 0.4169 - val_accuracy: 0.8081
Epoch 4/10
625/625 [==============================] - 300s 480ms/step - loss: 0.3782 - accuracy: 0.8328 - val_loss: 0.4014 - val_accuracy: 0.8245
Epoch 5/10
625/625 [==============================] - 305s 488ms/step - loss: 0.3171 - accuracy: 0.8609 - val_loss: 0.3842 - val_accuracy: 0.8391
Epoch 6/10
625/625 [==============================] - 286s 458ms/step - loss: 0.2527 - accuracy: 0.8921 - val_loss: 0.3926 - val_accuracy: 0.8343
Epoch 7/10
625/625 [==============================] - 308s 493ms/step - loss: 0.1913 - accuracy: 0.9215 - val_loss: 0.4100 - val_accura

In [7]:
model.save('model.h5')